In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from requests.exceptions import HTTPError
from random import randint
import time

# first download the list of words

In [2]:
def retrieve_webpage(req_url):
    """Get a webpage with the requests module & return the response"""
    user_agents = [
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"},
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"},
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0"},
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36"},
        {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"},
        {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"},
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"},
        {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.2 Safari/605.1.15"},
        {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15"},
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:91.0) Gecko/20100101 Firefox/91.0"},
        {'User-Agent': "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36"},
        {'User-Agent': "Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36"},
        {'User-Agent': "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1"},
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
    ]
    try:
        req_response = requests.get(req_url, headers=user_agents[randint(0, len(user_agents))])
        req_response.raise_for_status()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err} for URL {req_url}')
        return None
    except Exception as err:
        print(f'Other error occurred: {err} for URL {req_url}')
        return None
    else:
        # print('Request success!')
        return req_response

In [6]:
time.sleep(2.4) # in sec
url_wordlist = "https://www.oxfordlearnersdictionaries.com/wordlists/oxford3000-5000"
req_response = retrieve_webpage(url_wordlist)

if not req_response :
    print("fail")
elif str(req_response) == '<Response [200]>':
    print("success")
else:
    print("something wrong")

success


In [7]:
s = BeautifulSoup(req_response.content, 'html.parser')
all_elts = s.find_all("li")[34:-44]
words, levels, def_urls, types, sound_urls = [], [], [], [], []

for i in all_elts:
    i = str(i)
    if "data-hw" in i:
        words.append(i.split('data-hw="')[1].split('"')[0])
    else:
        words.append("No word")
    if "data-ox5000" in i:
        levels.append(i.split('data-ox5000="')[1].split('"')[0])
    else:
        levels.append("No level")
    if "href" in i:
        def_urls.append(i.split('href="')[1].split('"')[0])
    else:
        def_urls.append("No def_url")
    if 'class="pos' in i:
        types.append(i.split('class="pos">')[1].split('<')[0])
    else:
        types.append("No types")
    if "data-src-mp3" in i:
        sound_urls.append(i.split('pron-us" data-src-mp3="')[1].split('"')[0])
    else:
        sound_urls.append("No sound url")
    

len(words), len(levels), len(def_urls), len(types), len(sound_urls)

(5948, 5948, 5948, 5948, 5948)

In [8]:
df = pd.DataFrame(list(zip(words, levels, def_urls, types, sound_urls)), columns =['words', 'levels', 'def_urls', 'types', 'sound_urls'])
df.to_csv('words_list', index=False)
df.head(3)

,words,levels,def_urls,types,sound_urls
0,a,a1,/definition/english/a_1,indefinite article,/media/english/us_pron/a/a__/a__us/a__us_2_rr.mp3
1,abandon,b2,/definition/english/abandon_1,verb,/media/english/us_pron/a/aba/aband/abandon__us...
2,ability,a2,/definition/english/ability_1,noun,/media/english/us_pron/a/abi/abili/ability__us...


In [9]:
df.tail(3)

,words,levels,def_urls,types,sound_urls
5945,youth,b1,/definition/english/youth,noun,/media/english/us_pron/y/you/youth/youth__us_1...
5946,zero,a2,/definition/english/zero_1,number,/media/english/us_pron/z/zer/zero_/zero__us_1_...
5947,zone,b2,/definition/english/zone_1,noun,/media/english/us_pron/z/zon/zone_/zone__us_1.mp3


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5948 entries, 0 to 5947
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   words       5948 non-null   object
 1   levels      5948 non-null   object
 2   def_urls    5948 non-null   object
 3   types       5948 non-null   object
 4   sound_urls  5948 non-null   object
dtypes: object(5)
memory usage: 232.5+ KB


In [13]:
df[df['levels'] == 'No level'].shape[0]

5

In [18]:
df[df['levels'] == 'No level']

,words,levels,def_urls,types,sound_urls
47,accounting,No level,/definition/english/accounting,noun,/media/english/us_pron/a/acc/accou/accounting_...
233,angrily,No level,/definition/english/angrily,adverb,/media/english/us_pron/a/ang/angri/angrily__us...
889,cleaning,No level,/definition/english/cleaning,noun,/media/english/us_pron/c/cle/clean/cleaning__u...
2058,feeding,No level,/definition/english/feeding,noun,/media/english/us_pron/f/fee/feedi/feeding__us...
3176,major,No level,/definition/english/major_2,noun,/media/english/us_pron/m/maj/major/major__us_2...


In [21]:
df['levels'].unique()

array(['a1', 'b2', 'a2', 'c1', 'b1', 'No level'], dtype=object)

In [30]:
df.loc[df[df['levels'] == 'No level'].index, 'levels'] = 'a1'

In [38]:
df[df['sound_urls'] == 'No sound url'].shape[0]

1

In [39]:
df[df['sound_urls'] == 'No sound url']

,words,levels,def_urls,types,sound_urls
3559,nursing,b2,/definition/english/nursing,noun,No sound url


In [44]:
df[df.duplicated()].shape[0]

0

# Let's clean & transform the data a little bit

In [47]:
df.head()

,words,levels,def_urls,types,sound_urls
0,a,a1,/definition/english/a_1,indefinite article,/media/english/us_pron/a/a__/a__us/a__us_2_rr.mp3
1,abandon,b2,/definition/english/abandon_1,verb,/media/english/us_pron/a/aba/aband/abandon__us...
2,ability,a2,/definition/english/ability_1,noun,/media/english/us_pron/a/abi/abili/ability__us...
3,able,a2,/definition/english/able_1,adjective,/media/english/us_pron/a/abl/able_/able__us_2.mp3
4,abolish,c1,/definition/english/abolish,verb,/media/english/us_pron/a/abo/aboli/abolish__us...


In [62]:
BASE_URL = "https://www.oxfordlearnersdictionaries.com"

df['def_urls'] = df['def_urls'].apply(lambda x: BASE_URL + x)
df['levels'] = df['levels'].apply(lambda x: x.upper())
df['types'] = df['types'].apply(lambda x: "(" + x + ")")
df['sound_urls'] = df['sound_urls'].apply(lambda x: BASE_URL + x)
df['sound_files'] = df['sound_urls'].apply(lambda x: x.split("/")[-1])
df.head()

,words,levels,def_urls,types,sound_urls,sound_files
0,a,A1,https://www.oxfordlearnersdictionaries.com/def...,(indefinite article),https://www.oxfordlearnersdictionaries.com/med...,a__us_2_rr.mp3
1,abandon,B2,https://www.oxfordlearnersdictionaries.com/def...,(verb),https://www.oxfordlearnersdictionaries.com/med...,abandon__us_2.mp3
2,ability,A2,https://www.oxfordlearnersdictionaries.com/def...,(noun),https://www.oxfordlearnersdictionaries.com/med...,ability__us_4.mp3
3,able,A2,https://www.oxfordlearnersdictionaries.com/def...,(adjective),https://www.oxfordlearnersdictionaries.com/med...,able__us_2.mp3
4,abolish,C1,https://www.oxfordlearnersdictionaries.com/def...,(verb),https://www.oxfordlearnersdictionaries.com/med...,abolish__us_1.mp3


# Retrieve the sound files

In [ ]:
!mkdir sounds

In [64]:
with open("./sounds" + "/" + "url_list.txt", "w") as outfile:
    outfile.write("\n".join(set(df['sound_urls'])))

In [ ]:
!cd sounds, wget -i url_list.txt

In [79]:
from os import listdir
from os.path import isfile, join

files_list = [f for f in listdir("./sounds") if isfile(join("./sounds", f))]
print(len(files_list))
print(files_list[:10])

5012
['test__us_1.mp3', 'nuclear__us_1.mp3', 'stay__us_1.mp3', 'representative__us_4.mp3', 'embarrassed__us_1.mp3', 'partly__us_1.mp3', 'administer__us_1.mp3', 'methodology__us_1.mp3', 'accordingly__us_1.mp3', 'proposition__us_1.mp3']


In [80]:
df.to_csv('word_list.csv', index=False)

In [ ]:
https://www.oxfordlearnersdictionaries.com/media/english/us_pron/a/a__/a__us/a__us_2_rr.mp3

In [ ]:
url_1 = "https://www.oxfordlearnersdictionaries.com/definition/english/absolute_1"
web_page_1 = retrieve_webpage(url_1)

In [ ]:
def get_data(resp):
    """Parse HTTP response of a single webpage with BS4 and return relevant data"""
    
    # -------------- add try except with nan -------------
    soup = BeautifulSoup(resp.content, 'html.parser')
    phonetic = soup.find_all("div", class_="phons_n_am")[0].find_all("span", class_="phon")[0].contents[0]
    senses = soup.find_all("li", class_="sense")
    definitions = [se.find_all(class_="def")[0].contents[0] for se in senses]
     
    examples = [] # a list of (list of examples for one definition)
    for se in senses:
        all_examples = se.find_all("ul", class_="examples")[0].find_all(htag="li")
        examples.append([e.contents[0].text for e in all_examples])
    
    return phonetic, definitions, examples


print(get_data(web_page_1))

In [ ]:
https://www.oxfordlearnersdictionaries.com/wordlists/oxford3000-5000

In [ ]:
url = "https://www.oxfordlearnersdictionaries.com/wordlists/oxford3000-5000"

try:
    headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:53.0) Gecko/20100101 Firefox/53.0"} # UserAgent().random}
    req_response = requests.get(url, headers=headers)
    req_response.raise_for_status()
except:
    result = "Nan"
else:      
    s = BeautifulSoup(req_response.content, 'html.parser')

In [ ]:
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:53.0) Gecko/20100101 Firefox/53.0"},
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:53.0) Gecko/20100101 Firefox/53.0"},
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:53.0) Gecko/20100101 Firefox/53.0"},
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:53.0) Gecko/20100101 Firefox/53.0"}